In [ ]:
import tensorflow as tf
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
dirt='../models/1706293064.2710369'

In [ ]:
with open(f'{dirt}/params.json', 'r') as f:
    params = json.load(f)

max_length = params['max_length']
padding_type = params['padding_type']
vocab_size = params['vocab_size']
embedding_dim = params['embedding_dim']
trunc_type = params['trunc_type']
oov_tok = params['oov_tok']
model_accuracy = params['model_accuracy']

In [ ]:
path=os.path.join(dirt,model_accuracy)

In [ ]:
model = tf.keras.models.load_model(f'{path}.keras')
history=json.load(open(f'{path}.json','r'))
word_index=json.load(open(f'../processed/word_index.json','r'))

In [ ]:
test_x = np.load('../processed/test_padded.npy')

In [ ]:
model.summary()

In [ ]:
test_x = test_x.reshape((test_x.shape[0], -1))
data=tf.data.Dataset.from_tensor_slices((test_x))
data=data.batch(32)
predictions, indices = [], []
for idx,x in tqdm(enumerate(data), leave=False):
    pred = model(x, training=False)
    # predictions.extend(tf.squeeze(pred).numpy().tolist())
    predictions.extend((tf.squeeze(pred)>0.5).numpy().tolist())
pred_df = pd.DataFrame({ 'pred': predictions})

In [ ]:
pred_df['ID']=pred_df.index
pred_df['pred'] = pred_df['pred'].astype(int)
pred_df=pred_df[['ID','pred']]

In [ ]:
pred_df.to_csv(f'{dirt}/prediction.csv',index=False)